In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline  
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold


from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import StackingClassifier

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, log_loss, roc_auc_score

import warnings
warnings.filterwarnings("ignore")


In [10]:
df = pd.read_csv(r'C:\Users\DAI.STUDENTSDC\Desktop\Machine Learning\Data Sets\Cases\Satellite Imaging\Satellite.csv', sep=';')
df.head()

,x.1,x.2,x.3,x.4,x.5,x.6,x.7,x.8,x.9,x.10,...,x.28,x.29,x.30,x.31,x.32,x.33,x.34,x.35,x.36,classes
0,92,115,120,94,84,102,106,79,84,102,...,104,88,121,128,100,84,107,113,87,grey soil
1,84,102,106,79,84,102,102,83,80,102,...,100,84,107,113,87,84,99,104,79,grey soil
2,84,102,102,83,80,102,102,79,84,94,...,87,84,99,104,79,84,99,104,79,grey soil
3,80,102,102,79,84,94,102,79,80,94,...,79,84,99,104,79,84,103,104,79,grey soil
4,84,94,102,79,80,94,98,76,80,102,...,79,84,103,104,79,79,107,109,87,grey soil


In [11]:
X = df.drop(['classes'],axis=1)
y = df['classes']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=24)

In [13]:
nb = GaussianNB()
knn = KNeighborsClassifier(n_neighbors=5) 
dtc = DecisionTreeClassifier(random_state=24)  
svm = SVC(probability=True, random_state=24)
lr = LogisticRegression(random_state=24)
rf = RandomForestClassifier(random_state=24)

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=24)

stack = StackingClassifier(
    estimators=[
        ('knn', knn),
        ('dtc', dtc),
        ('nb', nb),    
        ('svm', svm), 
    ],
)
params = [
    {
        'final_estimator': [lr],
        'passthrough': [True, False],
        'final_estimator__C': [0.01, 0.1, 1],
    }, 
    {
        'final_estimator': [rf],
        'passthrough': [True, False],
        'final_estimator__n_estimators': [5, 10 ],   
        'final_estimator__max_depth': [None, 5, 10],
        'final_estimator__max_features': [None, 'auto', 'sqrt', 'log2'],
        'final_estimator__min_samples_split': [2, 5, 10],
    }
]


grid = GridSearchCV(
    estimator=stack, 
    param_grid=params,
    cv=kfold, 
    scoring='neg_log_loss'
)


In [ ]:
grid.get_params

<bound method BaseEstimator.get_params of GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=StackingClassifier(estimators=[('knn',
                                                       KNeighborsClassifier()),
                                                      ('dtc',
                                                       DecisionTreeClassifier(random_state=24)),
                                                      ('nb', GaussianNB()),
                                                      ('svm',
                                                       SVC(probability=True,
                                                           random_state=24))]),
             param_grid=[{'final_estimator': [LogisticRegression(random_state=24)],
                          'final_estimator__C': [0.01, 0.1, 1],
                          'passthrough': [True, False]},
                         {'final_estimator': [RandomForestClassifier(random_state

In [15]:
grid.fit(X_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=StackingClassifier(estimators=[('knn',
                                                       KNeighborsClassifier()),
                                                      ('dtc',
                                                       DecisionTreeClassifier(random_state=24)),
                                                      ('nb', GaussianNB()),
                                                      ('svm',
                                                       SVC(probability=True,
                                                           random_state=24))]),
             param_grid=[{'final_estimator': [LogisticRegression(random_state=24)],
                          'final_estimator__C': [0.01, 0.1, 1],
                          'passthrough': [True, False]},
                         {'final_estimator': [RandomForestClassifier(random_state=24)],
                          'final_estimator__max_depth': [None, 5, 10],
                          'final_estimator__max_features': [None, 'auto',
                                                            'sqrt', 'log2'],
                          'final_estimator__min_samples_split': [2, 5, 10],
                          'final_estimator__n_estimators': [5, 10],
                          'passthrough': [True, False]}],
             scoring='neg_log_loss')

In [16]:
grid.best_score_, grid.best_params_

(-0.2622987375197809,
 {'final_estimator': LogisticRegression(random_state=24),
  'final_estimator__C': 1,
  'passthrough': False})

In [17]:
best_model = grid.best_estimator_

y_pred = best_model.predict(X_test)
accuracy_score(y_test, y_pred)

0.9067840497151735

In [18]:
tst_df = pd.read_csv(r'C:\Users\DAI.STUDENTSDC\Desktop\Machine Learning\Data Sets\Cases\Satellite Imaging\tst_satellite.csv')

In [19]:
import pickle
with open('best_model_satellite.pkl', 'wb') as f:
    pickle.dump(best_model, f)


with open('best_model_satellite.pkl', 'rb') as f:
    best_model = pickle.load(f)

pred = best_model.predict(tst_df)

pred

array(['grey soil', 'grey soil', 'grey soil', 'grey soil', 'grey soil',
       'grey soil', 'very damp grey soil', 'grey soil', 'grey soil',
       'grey soil', 'grey soil', 'grey soil', 'grey soil', 'grey soil',
       'grey soil', 'grey soil', 'grey soil', 'grey soil'], dtype=object)

In [30]:
!pip install gradio --user


  Using cached gradio-4.44.1-py3-none-any.whl (18.1 MB)
  Using cached typer-0.13.0-py3-none-any.whl (44 kB)
  Using cached tomlkit-0.12.0-py3-none-any.whl (37 kB)
  Using cached MarkupSafe-2.1.5-cp38-cp38-win_amd64.whl (17 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl (9.8 kB)
  Using cached rich-13.9.4-py3-none-any.whl (242 kB)
  Using cached pygments-2.18.0-py3-none-any.whl (1.2 MB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl (87 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl (10.0 kB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

spyder 4.1.5 requires pyqt5<5.13; python_version >= "3", which is not installed.
spyder 4.1.5 requires pyqtwebengine<5.13; python_version >= "3", which is not installed.
huggingface-hub 0.26.2 requires packaging>=20.9, but you'll have packag

In [2]:
!pip install markupsafe==2.0.1


ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'C:\\ProgramData\\Anaconda3\\Lib\\site-packages\\markupsafe\\__init__.py'
Consider using the `--user` option or check the permissions.



  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 2.1.5
    Uninstalling MarkupSafe-2.1.5:
      Successfully uninstalled MarkupSafe-2.1.5
  Rolling back uninstall of MarkupSafe
  Moving to c:\users\dai.studentsdc\appdata\roaming\python\python38\site-packages\markupsafe-2.1.5.dist-info\
   from C:\Users\DAI.STUDENTSDC\AppData\Roaming\Python\Python38\site-packages\~arkupsafe-2.1.5.dist-info
  Moving to c:\users\dai.studentsdc\appdata\roaming\python\python38\site-packages\markupsafe\
   from C:\Users\DAI.STUDENTSDC\AppData\Roaming\Python\Python38\site-packages\~arkupsafe


In [7]:
df = pd.read_csv(r'C:\Users\DAI.STUDENTSDC\Desktop\Machine Learning\Data Sets\Cases\Satellite Imaging\Satellite.csv', sep=';')
cols = df.columns

In [8]:
import gradio as gr 

import os 
os.chdir(r"C:\Users\DAI.STUDENTSDC\Desktop\Machine Learning\Lab Practical\15-Stacking")

def predict(cols):
    tst = pd.DataFrame([cols],
          columns=['RI', 'Na', 'Mg', 'Al', 'Si', 'K', 'Ca', 'Ba', 'Fe'])    
    filehandler = open("best_model_satellite.pkl", "rb")
    bm_loaded = pickle.load(filehandler)
    print(tst)
    return bm_loaded.predict(tst)[0] 
      

demo = gr.Interface(
    fn=predict,
    inputs=["number"] * len(cols),
    outputs=["text"]
)





demo.launch()



IMPORTANT: You are using gradio version 3.1.4, however version 4.44.1 is available, please upgrade.
--------
Running on local URL:  http://127.0.0.1:7862/

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x1569c7dd4c0>, 'http://127.0.0.1:7862/', None)